In [1]:
import pandas as pd
from gensim.models import LdaModel
from gensim import corpora
import pickle

In [2]:
data = pd.read_csv("data/cleaned_data1.csv")

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_4941/1751319474.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/cleaned_data1.csv")


## Load topic model, corpus and dictionary

In [3]:
model = LdaModel.load("lda_models/model30")

In [4]:
# Load corpus and dictionary
with open('corpus_and_dict.pkl', 'rb') as f:
    corpus, dictionary = pickle.load(f)

## Visualize topics

In [5]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(model, corpus, dictionary)

In [6]:
pyLDAvis.display(lda_viz)

In [ ]:
data["topic_distribution"] = [model.get_document_topics(bow) for bow in corpus]

In [ ]:
data["dominant_topic"] = [max(model.get_document_topics(bow), key=lambda x: x[1])[0] for bow in corpus]
data["prob_topic"] = [max(model.get_document_topics(bow), key=lambda x: x[1])[1] for bow in corpus]

## Inspect each topic's top documents

## Model 28

### Topic 0

This is a vague topic. Let's inspect it 

even, bad, best, well, hard, always, first, help, need, try, problem, everything, actually, everyone, sure, look, different, care, enough, advice

In [ ]:
topic0 = data[data.dominant_topic == 0].sort_values("prob_topic", ascending= False)
print(topic0["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic0[["type","cleaned_text", "prob_topic"]][0:20])

### Topic 1

now, never, back, year, right, old, ever, live, age, man, young, happy, kid, right_now, shit, alone, parent, last, month, family

In [ ]:
topic1 = data[data.dominant_topic == 1].sort_values("prob_topic", ascending= False)
print(topic1["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic1[["type","cleaned_text", "prob_topic"]][0:20])

## Model 30

### Topic 0

This is a vague topic. Let's inspect it 

even, now, never, right, bad, friend, well, back, hard, best, first, always, year, start, everything, long, new, relationship, happy, need

In [ ]:
topic0 = data[data.dominant_topic == 0].sort_values("prob_topic", ascending= False)
print(topic0["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic0[["type","cleaned_text", "prob_topic"]][0:20])